# Diabetes prediction with synthea data

###### Mostly from https://github.com/IBM/example-health-machine-learning/blob/master/diabetes-prediction.ipynb

### Import data in pandas dataframes 

In [1]:
import pandas as pd 
import numpy as np

#load data into pandas dataframes
data_dir = "../../data/synthea/"
conditions_file = data_dir+"conditions.csv"
medications_file = data_dir+"medications.csv"
observatios_file = data_dir+"observations.csv"
patients_file = data_dir+"patients.csv"

df_cond = pd.read_csv(conditions_file)
df_med = pd.read_csv(medications_file)
df_obs = pd.read_csv(observatios_file)
df_pat = pd.read_csv(patients_file)

In [2]:
df_obs.head()

,DATE,PATIENT,ENCOUNTER,CODE,DESCRIPTION,VALUE,UNITS,TYPE
0,2011-03-31T02:00:17Z,7d3e489a-7789-9cd6-2a1b-711074af481b,814174f3-2e0e-1625-de48-9c40732c9149,8302-2,Body Height,167.0,cm,numeric
1,2011-03-31T02:00:17Z,7d3e489a-7789-9cd6-2a1b-711074af481b,814174f3-2e0e-1625-de48-9c40732c9149,72514-3,Pain severity - 0-10 verbal numeric rating [Sc...,3.0,{score},numeric
2,2011-03-31T02:00:17Z,7d3e489a-7789-9cd6-2a1b-711074af481b,814174f3-2e0e-1625-de48-9c40732c9149,29463-7,Body Weight,71.1,kg,numeric
3,2011-03-31T02:00:17Z,7d3e489a-7789-9cd6-2a1b-711074af481b,814174f3-2e0e-1625-de48-9c40732c9149,39156-5,Body Mass Index,25.5,kg/m2,numeric
4,2011-03-31T02:00:17Z,7d3e489a-7789-9cd6-2a1b-711074af481b,814174f3-2e0e-1625-de48-9c40732c9149,59576-9,Body mass index (BMI) [Percentile] Per age and...,83.6,%,numeric


In [3]:
df_pat.head()

,Id,BIRTHDATE,DEATHDATE,SSN,DRIVERS,PASSPORT,PREFIX,FIRST,LAST,SUFFIX,...,BIRTHPLACE,ADDRESS,CITY,STATE,COUNTY,ZIP,LAT,LON,HEALTHCARE_EXPENSES,HEALTHCARE_COVERAGE
0,7d3e489a-7789-9cd6-2a1b-711074af481b,1993-01-28,NaN,999-95-8631,S99916705,X24646789X,Mr.,Jon665,Pacocha935,NaN,...,Lawrence Massachusetts US,942 Fahey Overpass Apt 21,Natick,Massachusetts,Middlesex County,NaN,42.309347,-71.349633,569019.69,2293.12
1,a3795ec8-54f3-e99e-a4b1-4c067f3141d7,1971-12-01,NaN,999-62-4431,S99941017,X38787090X,Mr.,Dick869,Streich926,NaN,...,Swansea Massachusetts US,1064 Hickle View Apt 7,Chicopee,Massachusetts,Hampden County,1020.0,42.198239,-72.554752,18755.46,0.00
2,3829c803-1f4c-74ed-0d8f-36e502cadd0f,2005-01-07,NaN,999-21-2332,NaN,NaN,NaN,Cordell41,Eichmann909,NaN,...,Chelmsford Massachusetts US,560 Ritchie Way Suite 68,Swansea,Massachusetts,Bristol County,NaN,41.748125,-71.182914,361770.00,2768.96
3,d7acfddb-f4c2-69f4-2081-ad1fb8490448,1990-07-04,NaN,999-53-1990,S99932677,X67053099X,Mrs.,Cheri871,Oberbrunner298,NaN,...,Cambridge Massachusetts US,268 Hansen Loaf Apt 62,Lowell,Massachusetts,Middlesex County,1850.0,42.662520,-71.368933,703332.77,5551.19
4,474766f3-ee93-f5d6-84c3-db38ba803394,2012-04-03,NaN,999-57-2653,NaN,NaN,NaN,Desmond566,O'Conner199,NaN,...,Cohasset Massachusetts US,831 Schumm Lock Apt 62,Westborough,Massachusetts,Worcester County,NaN,42.253951,-71.563825,206450.27,2284.86


### Feature selection

Select the features of interests: 

- Systolic blood pressure readings from the observations (code 8480-6).
- Select diastolic blood pressure readings (code 8462-4).
- Select HDL cholesterol readings (code 2085-9).
- Select LDL cholesterol readings (code 18262-6).
- Select BMI (body mass index) readings (code 39156-5).


In [4]:
def feature_selection_obs(df, code):
    return df[df["CODE"]==code][["PATIENT", "DATE", "VALUE"]].drop_duplicates().reset_index(drop=True)

#select feautures from observations
df_systolic = feature_selection_obs(df_obs, "8480-6").rename(columns={"VALUE": "SYSTOLIC_BP"})
df_diastolic = feature_selection_obs(df_obs, "8462-4").rename(columns={"VALUE": "DIASTOLIC_BP"})
df_hdl = feature_selection_obs(df_obs, "2085-9").rename(columns={"VALUE": "HDL"})
df_ldl = feature_selection_obs(df_obs, "18262-6").rename(columns={"VALUE": "LDL"})
df_bmi = feature_selection_obs(df_obs, "39156-5").rename(columns={"VALUE": "BMI"})

In [5]:
len(df_systolic), len(df_diastolic), len(df_hdl), len(df_ldl), len(df_bmi)

(83540, 83541, 26900, 26900, 57880)

Merge the dataframes (inner join for now, to avoid dealing with missing values)

In [6]:
df1 = pd.merge(df_systolic, df_diastolic, on=["PATIENT", "DATE"], how='inner')
df2 = pd.merge(df1, df_hdl, on=["PATIENT", "DATE"], how='inner')
df3 = pd.merge(df2, df_ldl, on=["PATIENT", "DATE"], how='inner')
df4 = pd.merge(df3, df_bmi, on=["PATIENT", "DATE"], how='inner')

In [7]:
len(df4)

21224

In [8]:
df4.head()

,PATIENT,DATE,SYSTOLIC_BP,DIASTOLIC_BP,HDL,LDL,BMI
0,a3795ec8-54f3-e99e-a4b1-4c067f3141d7,2013-01-16T22:06:58Z,128.0,88.0,64.5,89.2,22.4
1,a3795ec8-54f3-e99e-a4b1-4c067f3141d7,2017-12-20T22:06:58Z,116.0,71.0,64.9,78.0,22.4
2,9bafdf36-6e60-e93e-7925-c8d15a49ea62,2012-11-25T09:32:01Z,125.0,82.0,72.9,97.3,27.6
3,9bafdf36-6e60-e93e-7925-c8d15a49ea62,2015-12-13T09:32:01Z,104.0,89.0,64.3,71.3,27.6
4,9bafdf36-6e60-e93e-7925-c8d15a49ea62,2018-12-30T09:32:01Z,121.0,77.0,61.2,77.8,27.6


Join also the age (derived from birth date in PATIENT dataset)

In [9]:
df5 = pd.merge(df4, df_pat[["Id", "BIRTHDATE"]].rename(columns={"Id": "PATIENT"}), on =["PATIENT"], how='inner')

In [10]:
df5["DATE"] = [x.split("T")[0] for x in list(df5["DATE"])]

In [11]:
from datetime import datetime
#from https://stackoverflow.com/questions/8419564/difference-between-two-dates-in-python
def days_between(d1, d2):
    d1 = datetime.strptime(d1, "%Y-%m-%d")
    d2 = datetime.strptime(d2, "%Y-%m-%d")
    return abs((d2 - d1).days)

def age_calculation(l1, l2):
    age_list = []
    i = 0
    for i in range(0, len(l1)):
        age_list.append(days_between(l1[i], l2[i]) / 365.00)
    return age_list

df5["AGE"] = age_calculation(list(df5["DATE"]), list(df5["BIRTHDATE"]))

In [12]:
df5.drop(["BIRTHDATE"], axis=1, inplace=True)

In [13]:
df5.head()

,PATIENT,DATE,SYSTOLIC_BP,DIASTOLIC_BP,HDL,LDL,BMI,AGE
0,a3795ec8-54f3-e99e-a4b1-4c067f3141d7,2013-01-16,128.0,88.0,64.5,89.2,22.4,41.156164
1,a3795ec8-54f3-e99e-a4b1-4c067f3141d7,2017-12-20,116.0,71.0,64.9,78.0,22.4,46.084932
2,9bafdf36-6e60-e93e-7925-c8d15a49ea62,2012-11-25,125.0,82.0,72.9,97.3,27.6,58.167123
3,9bafdf36-6e60-e93e-7925-c8d15a49ea62,2015-12-13,104.0,89.0,64.3,71.3,27.6,61.216438
4,9bafdf36-6e60-e93e-7925-c8d15a49ea62,2018-12-30,121.0,77.0,61.2,77.8,27.6,64.265753


Now, we find the patient with diabetes diagnosis, and select the start date column (equivalent to first diagnosis), in the CONDITION dataset. 

In [14]:
df_pat_diab = df_cond[df_cond.DESCRIPTION == "Diabetes"][["PATIENT", "START"]]

In [15]:
df6 = pd.merge(df5, df_pat_diab, on=["PATIENT"], how="left")

In [16]:
df6["HAS_DIABETES"] = [(0 if (type(el) == float and np.isnan(el)) else 1) for el in list(df6["START"])]

In [17]:
df6.head()

,PATIENT,DATE,SYSTOLIC_BP,DIASTOLIC_BP,HDL,LDL,BMI,AGE,START,HAS_DIABETES
0,a3795ec8-54f3-e99e-a4b1-4c067f3141d7,2013-01-16,128.0,88.0,64.5,89.2,22.4,41.156164,NaN,0
1,a3795ec8-54f3-e99e-a4b1-4c067f3141d7,2017-12-20,116.0,71.0,64.9,78.0,22.4,46.084932,NaN,0
2,9bafdf36-6e60-e93e-7925-c8d15a49ea62,2012-11-25,125.0,82.0,72.9,97.3,27.6,58.167123,NaN,0
3,9bafdf36-6e60-e93e-7925-c8d15a49ea62,2015-12-13,104.0,89.0,64.3,71.3,27.6,61.216438,NaN,0
4,9bafdf36-6e60-e93e-7925-c8d15a49ea62,2018-12-30,121.0,77.0,61.2,77.8,27.6,64.265753,NaN,0


### Filter data
For this example, we filter the positive observations taken before diagnosis and then we reduce the observations to a single one per patient. In the future, it might be valuable keeping the time and try to predict diabetes before it occurs (e.g. with RNN). In this case, however, we need to check better the generative model underlying synthea, as in the notebook we are trying to reproduce here: "The impact of the condition (diabetes) is not reflected in the observations until the patient is diagnosed with the condition in a wellness visit". However, there is a condition called "Prediabetes" which we could take into account. 

In [18]:
def date_to_int(string_date):
    #a date can also be nan (float type)
    return int(string_date.replace("-", "")) if type(string_date) == str else 0
def col_date_to_int(col_date):
    return list(map(date_to_int, col_date))

df6["temp_date"] = col_date_to_int(df6["DATE"])
df6["temp_start"] = col_date_to_int(df6["START"])

In [19]:
df_with_diab = df6[df6.HAS_DIABETES == 1]
df_to_discard = df_with_diab[df_with_diab["temp_start"] > df_with_diab["temp_date"]]
len(df_to_discard)

57

In [20]:
df7 = df6.drop(index = df_to_discard.index, inplace=False).reset_index().drop(columns=["index"])

Let's now reduce the observations to a single observation per patient (the earliest available observation)

In [21]:
df7.sort_values(by=["temp_date"], axis=0, inplace=True)
df7.reset_index(inplace=True)
df7 = df7.drop(columns="index")
df7["OBS_INDEX"] = df7.groupby(["PATIENT"]).cumcount()+1
df8 = df7[df7.OBS_INDEX == 1]
df8.reset_index(inplace=True)
df8 = df8.drop(columns=["index", "temp_date", "temp_start", "OBS_INDEX", "START", "DATE"])
df8

,PATIENT,SYSTOLIC_BP,DIASTOLIC_BP,HDL,LDL,BMI,AGE,HAS_DIABETES
0,28de1ba3-efdc-1797-5a32-d4f8d3be0936,109.0,78.0,75.1,74.4,25.2,30.356164,0
1,6686010d-3d7b-d69d-d2cd-8bbe4b3e6041,160.0,104.0,62.1,82.6,28.2,30.356164,0
2,42217d99-02e1-2cc8-83ed-5101c246a559,112.0,84.0,69.2,94.6,30.1,31.221918,0
3,c429d985-225b-f380-4462-57852cf61186,121.0,74.0,63.1,109.0,29.6,31.221918,0
4,06365dfa-6203-5413-2c6d-553a4a988c1f,117.0,82.0,64.6,94.1,34.3,34.232877,0
...,...,...,...,...,...,...,...,...
3821,391d2527-19ca-8b83-2cd0-5c484946b2b7,156.0,95.0,76.5,76.4,34.1,30.358904,0
3822,964603c6-e5bd-6d86-5b21-34056a4a651a,124.0,78.0,68.0,102.1,25.3,31.221918,0
3823,3d44d71e-2241-29c2-6aed-7f7f4d65d8d4,123.0,84.0,75.0,66.5,30.0,30.358904,0
3824,bbcfa21f-caa2-4540-d2b6-929f48ae27c2,130.0,71.0,69.1,99.3,29.3,31.221918,0


In [22]:
list(df8["HAS_DIABETES"]).count(0), list(df8["HAS_DIABETES"]).count(1)

(3480, 346)

In [23]:
#prepare dataset for pytorch
df9 = df8.drop(columns="PATIENT")

In [24]:
list(df9.columns)

['SYSTOLIC_BP', 'DIASTOLIC_BP', 'HDL', 'LDL', 'BMI', 'AGE', 'HAS_DIABETES']

### Divide into training and test set, and define train and test dataloader

In [42]:
df_input = df9.drop(columns="HAS_DIABETES")
df_y = df9["HAS_DIABETES"]

for col in list(df_input.columns):
    df_input[col] = list(map(float, df_input[col]))
    
train_ratio = 0.7

msk = np.random.rand(len(df9)) < train_ratio
train_set = df_input[msk].values
train_labels = df_y[msk].values
test_set = df_input[~msk].values
test_labels = df_y[~msk].values

In [121]:
import torch 

train_target = torch.tensor(train_labels.astype(np.float32))
train = torch.tensor(train_set.astype(np.float32)) 

test_target = torch.tensor(test_labels.astype(np.float32))
test = torch.tensor(test_set.astype(np.float32)) 

bs = 10
train_tensor = torch.utils.data.TensorDataset(train, train_target) 
train_loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = bs, shuffle = True)

test_tensor = torch.utils.data.TensorDataset(test, test_target) 
test_loader = torch.utils.data.DataLoader(dataset = test_tensor, batch_size = bs, shuffle = False)

### Simple logistic regression model

In [122]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x)
        return outputs

In [123]:
epochs = 10
input_dim = 6
output_dim = 2
lr_rate = 0.001

model = LogisticRegression(input_dim, output_dim)
criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate)

### Training loop and evaluation

In [177]:
from torch.autograd import Variable
iter = 0
loss_v = []
for epoch in range(int(epochs)):
    for i, (point, labels) in enumerate(train_loader):
        points = Variable(point.view(-1, 6))
        labels = Variable(labels.type(torch.LongTensor))
        
        optimizer.zero_grad()
        outputs = model(points)
        loss = criterion(outputs, labels)
        loss_v.append(loss.detach().numpy())
        loss.backward()
        optimizer.step()
        
        iter+=1
        if iter%200==0:
            # calculate Accuracy
            correct = 0
            total = 0
            for points, labels in test_loader:
                points = Variable(points.view(-1, 6))
                outputs = model(points)
                _, predicted = torch.max(outputs.data, 1)
                total+= labels.size(0)
                correct+= (predicted == labels).sum()
            accuracy = 100 * float(correct)/float(total)
            print("Epoch: {}. Iteration: {}. Loss: {}. Accuracy: {}.".format(epoch, iter, loss.item(), accuracy))

Epoch: 0. Iteration: 200. Loss: 7.707485929131508e-05. Accuracy: 90.78590785907859.
Epoch: 1. Iteration: 400. Loss: 0.0004317985731177032. Accuracy: 94.76061427280939.
Epoch: 2. Iteration: 600. Loss: 0.0. Accuracy: 90.60523938572719.
Epoch: 2. Iteration: 800. Loss: 1.2545890808105469. Accuracy: 45.347786811201445.
Epoch: 3. Iteration: 1000. Loss: 0.07240111380815506. Accuracy: 94.03794037940379.
Epoch: 4. Iteration: 1200. Loss: 0.0018771663308143616. Accuracy: 93.13459801264679.
Epoch: 5. Iteration: 1400. Loss: 1.3109453916549683. Accuracy: 79.67479674796748.
Epoch: 5. Iteration: 1600. Loss: 1.7740905284881592. Accuracy: 92.6829268292683.
Epoch: 6. Iteration: 1800. Loss: 0.0184627752751112. Accuracy: 94.2186088527552.
Epoch: 7. Iteration: 2000. Loss: 0.6591505408287048. Accuracy: 75.51942186088527.
Epoch: 8. Iteration: 2200. Loss: 1.10377836227417. Accuracy: 90.60523938572719.
Epoch: 8. Iteration: 2400. Loss: 0.2253536880016327. Accuracy: 92.773261065944.
Epoch: 9. Iteration: 2600. Los

In [141]:
#final test
labels = test_target
outputs = model(test)
_, predicted = torch.max(outputs.data,1)
total = labels.size(0)
correct = (predicted == labels).sum()
accuracy = float(correct)/float(total)

In [166]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn

cm = confusion_matrix(labels, predicted, labels=[0,1])
tn, fp, fn, tp = cm.ravel()

In [170]:
cm

array([[1002,    0],
       [ 105,    0]])

In [171]:
tn, fp, fn, tp

(1002, 0, 105, 0)